In [1]:

import glob
import tqdm
import torch
import json
import os
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import tensorboard


# import package

# model
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
from torch import optim

# dataset and transformation
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import os

# display images
from torchvision import utils
import matplotlib.pyplot as plt
%matplotlib inline

# utils
import numpy as np
from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter
import time
import copy

from model import *
from dataset import *

In [9]:
transform = transforms.Compose([transforms.ToTensor(),])
dataset = KwenDataset(path = 'dataset', transform= transform, lstm = True)
dataloader =DataLoader(dataset=dataset,batch_size=32,shuffle=True,drop_last=False)
def mobilenet(alpha=1, num_classes=1):
    return MobileNet(alpha, num_classes)

decive = 'cuda'
writer = SummaryWriter()
losss = {}
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = mobilenet().to(device)

transform = transforms.Compose([transforms.ToTensor(),])
dataset = KwenDataset(path = 'dataset', transform= transform, lstm = True)
dataloader =DataLoader(dataset=dataset,batch_size=20,shuffle=True,drop_last=False)

#criterion = nn.CrossEntropyLoss().to(device)
criterion = nn.MSELoss().to(device)
learning_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


count_idx = 0

for epoch in range(1):
    model.train()
    print(f"epoch : {epoch} ")
    for batch in dataloader:
        img = batch[0]
        wqi =batch[1]
        label = batch[2]
        
        label = label.to(torch.float32)
        label = label.to(device)
        
        x_= torch.stack(wqi).to(device)
        x_ = torch.transpose(x_,0,1)
        x_ = x_.to(torch.float32)

        x = img
        x = x.to(device)

        data = [x, x_]
        output = model(data)

        loss = criterion(output, label)
        
        loss = loss.to(torch.float32)
        loss.backward()
        
        optimizer.step()
        count_idx += 1
        writer.add_scalar('log', loss, count_idx)
        losss[count_idx] = loss.item()
        print("lr: ", optimizer.param_groups[0]['lr'])
        print('Loss: ', loss)
        print('output: ', output[0:2])
        if count_idx%10 == 1 :
            optimizer.param_groups[0]['lr'] *= 0.995

epoch : 0 
1


c:\Users\tlseh\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([20])) that is different to the input size (torch.Size([20, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


lr:  0.001
Loss:  tensor(7448.6763, device='cuda:0', grad_fn=<MseLossBackward0>)
output:  tensor([[-6.2769e-06],
        [ 9.1177e-05]], device='cuda:0', grad_fn=<SliceBackward0>)
1
lr:  0.000995
Loss:  tensor(6754.6812, device='cuda:0', grad_fn=<MseLossBackward0>)
output:  tensor([[0.0162],
        [0.0170]], device='cuda:0', grad_fn=<SliceBackward0>)
1
lr:  0.000995
Loss:  tensor(6822.6992, device='cuda:0', grad_fn=<MseLossBackward0>)
output:  tensor([[0.0412],
        [0.0517]], device='cuda:0', grad_fn=<SliceBackward0>)
1
lr:  0.000995
Loss:  tensor(7553.4966, device='cuda:0', grad_fn=<MseLossBackward0>)
output:  tensor([[0.0818],
        [0.0902]], device='cuda:0', grad_fn=<SliceBackward0>)
1
lr:  0.000995
Loss:  tensor(7505.0024, device='cuda:0', grad_fn=<MseLossBackward0>)
output:  tensor([[0.1414],
        [0.1472]], device='cuda:0', grad_fn=<SliceBackward0>)
1
lr:  0.000995
Loss:  tensor(7371.4819, device='cuda:0', grad_fn=<MseLossBackward0>)
output:  tensor([[0.2441],
       

In [33]:
img = batch[0]
wqi =batch[1]

x_= torch.stack(wqi).to(device)
x_ = torch.transpose(x_,0,1)
x_ = x_.to(torch.float32)

x = img
x = x.to(device)

data = [x, x_]
result = model(data)

1
torch.Size([20, 16])
torch.Size([20, 8])


In [36]:
result

tensor([[4.1963e-05],
        [5.1779e-04],
        [4.5688e-04],
        [4.7213e-04],
        [1.3451e-04],
        [5.0552e-04],
        [3.7219e-04],
        [5.8095e-04],
        [5.6967e-04],
        [4.7185e-04],
        [2.5451e-04],
        [8.7393e-04],
        [6.9446e-04],
        [5.5449e-04],
        [4.7565e-04],
        [3.5057e-04],
        [3.1085e-04],
        [3.6410e-04],
        [5.7352e-04],
        [5.9448e-04]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [20]:
i.size()

torch.Size([20, 16])

In [21]:
t.size()

torch.Size([20, 8])

In [29]:

torch.cat((i, t), dim=1).size()

torch.Size([20, 24])

In [23]:
i

tensor([[ 0.0177,  0.0094,  0.0121, -0.0026,  0.0048, -0.0031,  0.0027, -0.0009,
          0.0031, -0.0010, -0.0042, -0.0099,  0.0093, -0.0033, -0.0023, -0.0021],
        [ 0.0170,  0.0107,  0.0082, -0.0009,  0.0144, -0.0100, -0.0051,  0.0061,
          0.0035, -0.0075,  0.0014, -0.0095,  0.0001, -0.0052, -0.0042, -0.0060],
        [ 0.0176,  0.0072,  0.0102,  0.0055,  0.0057, -0.0077,  0.0006,  0.0052,
         -0.0027, -0.0033, -0.0010, -0.0034,  0.0082, -0.0048, -0.0102, -0.0151],
        [ 0.0146,  0.0103,  0.0104,  0.0104,  0.0069, -0.0081,  0.0056,  0.0133,
         -0.0034, -0.0109, -0.0044, -0.0079,  0.0084, -0.0097, -0.0057, -0.0107],
        [ 0.0034,  0.0064,  0.0116,  0.0002,  0.0056, -0.0226,  0.0018,  0.0054,
         -0.0006, -0.0088,  0.0039, -0.0047,  0.0014, -0.0018, -0.0045, -0.0003],
        [ 0.0107,  0.0081,  0.0074, -0.0023,  0.0078, -0.0123, -0.0035,  0.0013,
          0.0027, -0.0036, -0.0042, -0.0061,  0.0088, -0.0057, -0.0018, -0.0064],
        [ 0.0142,  0.0

In [24]:
t

tensor([[-0.0343,  0.0037,  0.0182, -0.0188, -0.0410,  0.0033, -0.0208, -0.0016],
        [-0.0354,  0.0016,  0.0183, -0.0185, -0.0417,  0.0038, -0.0211, -0.0024],
        [-0.0334,  0.0038,  0.0177, -0.0189, -0.0409,  0.0029, -0.0205, -0.0021],
        [-0.0551, -0.0210,  0.0244, -0.0115, -0.0482,  0.0177, -0.0244, -0.0041],
        [-0.0520, -0.0155,  0.0354, -0.0155, -0.0565,  0.0070, -0.0301,  0.0089],
        [-0.0542,  0.0335,  0.0475, -0.0143, -0.0159,  0.0122, -0.0042,  0.0054],
        [-0.0355,  0.0037,  0.0188, -0.0185, -0.0410,  0.0040, -0.0213, -0.0008],
        [-0.0343,  0.0038,  0.0182, -0.0187, -0.0409,  0.0034, -0.0208, -0.0015],
        [-0.0367,  0.0005,  0.0306, -0.0149, -0.0495,  0.0028, -0.0267,  0.0136],
        [-0.0349,  0.0038,  0.0185, -0.0186, -0.0409,  0.0037, -0.0211, -0.0011],
        [-0.0358, -0.0003,  0.0181, -0.0188, -0.0423,  0.0038, -0.0207, -0.0040],
        [-0.0558, -0.0045,  0.0281, -0.0106, -0.0459,  0.0081, -0.0094, -0.0177],
        [-0.0333